In [0]:
import os
import torch
import torchvision
import torch.utils.data as utils
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import timeit
import random

import matplotlib.pyplot as plt
import numpy as np

def imageBatch(nb_image):
    imgBatchR_T = torch.randint(0,255,(nb_image,3,9,9))
    imgBatchL_T = imgBatchR_T
    labelT = torch.ones(nb_image,1)

    imgBatchR_F = torch.randint(0,255,(nb_image,3,9,9)) #
    imgBatchL_F = torch.randint(0,255,(nb_image,3,9,9))
    labelF = torch.zeros(nb_image,1)

    finalR = torch.cat((imgBatchR_T,imgBatchR_F))
    finalL = torch.cat((imgBatchL_T,imgBatchL_F))
    finalLabel = torch.cat((labelT,labelF))

    return finalR, finalL, finalLabel

def firstStageCNN():
    return nn.Sequential(nn.Linear(3*9*9, 50), #L1
                         nn.ReLU(inplace=True),
                         
                         nn.Linear(50, 50), #L2
                         nn.ReLU(inplace=True))

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        self.cnn1 = firstStageCNN()
        
        self.cnn2 = firstStageCNN()

        
        self.fc = nn.Sequential(nn.Linear(100, 75), #L3
                                nn.ReLU(inplace=True),
                                
                                nn.Linear(75, 50), #L4
                                nn.ReLU(inplace=True),
                            
                                nn.Linear(50, 25), #L5
                                nn.ReLU(inplace=True),
                                
                                nn.Linear(25, 1), #L6
                                nn.Sigmoid())
    
    def forward(self, input1, input2):
        
        output1 = self.cnn1(input1.float().view(-1,3*9*9))
        output2 = self.cnn2(input2.float().view(-1,3*9*9))


        combined = torch.cat((output1.view(output1.size(0), -1),
                              output2.view(output2.size(0), -1)), dim=1)

        combined = torch.unsqueeze(combined,2)
        combined = torch.unsqueeze(combined,3)
        combined = combined.view(-1,100)
        
        out = self.fc(combined)
        
        return out

def train(net, finalR, finalL, finalLabel, EPOCHS, BATCH_SIZE):
    height, width = 9,9
    optimizer = optim.Adam(net.parameters(), lr=0.005)
    loss_function = nn.BCELoss()
    dataset = utils.TensorDataset(finalL, finalR, finalLabel)
    train_dataloader = DataLoader(dataset, shuffle=True, num_workers=8, batch_size=6)
    COUNTER = 1
    net.zero_grad()

    print("train function was executed")
    for epoch in range(EPOCHS):
        avg_loss = 0
        for i, data in enumerate(train_dataloader):
            img1, img2, depthmap = data
            # img1, img2, depthmap = img1.cuda(), img2.cuda(), depthmap.cuda()
            optimizer.zero_grad() # reset gradient
            outputs = net(img1, img2)
            outputs = outputs.cpu()
            depthmap = depthmap.cpu()
            loss = loss_function(outputs, depthmap)
            #print("Loss:", float(loss))
            avg_loss+=loss
            loss.backward()
            optimizer.step()
        #Print out images and epoch numbers 
        print("Epoch number: ", COUNTER)
        COUNTER += 1 
        # avg_loss = np.array(avg_loss)
        #print("Average Loss:", avg_loss/i)
        # plot_weights(net.cpu(), 0, single_channel = False)
        outputs = outputs.cpu()
        # img1 = img1.cpu()
        # img2 = img2.cpu()
        depthmap = depthmap.cpu()
        #print("Output size", outputs.size())
        #for param in net.parameters():
        #    print('average weight per layer: ', np.mean(np.array(param.data.cpu())))
        image = np.swapaxes(img1.numpy(), 1,3)
        image1 = np.swapaxes(img2.numpy(), 1,3)
        plt.figure()
        plt.imshow(image[0])
        plt.figure()
        plt.imshow(image1[0])
        plt.show()
        print('Likeliness value:', outputs[-1])
        print("Actual Likeness", depthmap[-1])
    return net

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/Colab Notebooks/Image 1"
!cp "/content/drive/My Drive/Colab Notebooks/Image 1/cropped_noisy_compilation_1.npy" "cropped_noisy_compilation_1.npy"
!cp "/content/drive/My Drive/Colab Notebooks/Image 1/cropped_original_compilation_1.npy" "cropped_original_compilation_1.npy"
!cp "/content/drive/My Drive/Colab Notebooks/Image 1/similarity_rating_compilation_1.npy" "similarity_rating_compilation_1.npy"

cropped_noisy_compilation_1.npy      test1.jpg
cropped_original_compilation_1.npy   test1_noisy.png
similarity_rating_compilation_1.npy


In [0]:
import matplotlib.pyplot as plt

NOISY = np.load("cropped_noisy_compilation_1.npy")
ORIG = np.load("cropped_original_compilation_1.npy")
LABEL = np.load("similarity_rating_compilation_1.npy")


# NOISY = np.swapaxes(NOISY, 1,3)
# for i in range(50):
#   plt.figure()
#   plt.imshow(NOISY[i])
#   plt.show()
# NOISY = np.swapaxes(NOISY, 1,3)


NOISY = torch.from_numpy(NOISY).float()
ORIG = torch.from_numpy(ORIG).float()
LABEL = torch.from_numpy(LABEL).float()

net = SiameseNetwork()
net#.cuda()
NumberIMG = 500
EPOCHS = 50

#finalR, finalL, finalLabel = imageBatch(NumberIMG)

In [0]:
final = train(net,NOISY,ORIG,LABEL,EPOCHS,NumberIMG)